In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import astropy.units as u
import astropy.coordinates as coord
import astropy.constants as const


cantat = pd.read_csv('Data\\CantatGaudin\\cantatgaudinfile.csv')
cantat = cantat.assign(r50_dist = (np.tan(cantat['r50']*np.pi/180) * cantat['dmode']))
cantat = cantat.assign(inner = np.where(cantat['Plx'] > 1.1, 1, 0))
hunt = pd.read_csv('Data\\Hunt\\huntfile.csv').query('Type == "o"')
xmatch = pd.read_csv('Data\\Hunt\\xmatchfile.csv').dropna(subset='Sep')
khar = pd.read_csv('Data\\Kharchenko\\kharchenkofile.csv').query('Type != "g"')
dias = pd.read_csv('Data\\Dias\\diasfile.csv')
dias['Cluster'] = dias['Cluster'].str.replace(' ', '_').str.replace('-', '_')


FileNotFoundError: [Errno 2] No such file or directory: 'Data\\CantatGaudin\\cantatgaudinfile.csv'

In [ ]:
cantat_xm = xmatch.query('SourceCat == "Cantat-Gaudin+20"')
khar_xm = xmatch.query('SourceCat == "Kharchenko+13"')

def datahandler(df_lit, df=hunt, crossmatch=xmatch):

    if df_lit is cantat: # Define the source catalog and the column name of the cluster name, each table has different names for the cluster name column
        sourcecat = 'Cantat-Gaudin+20'
        NameCol = 'Cluster'
    elif df_lit is khar:
        sourcecat = 'Kharchenko+13'
        NameCol = 'Name'
    elif df_lit is dias:
        sourcecat = 'Dias+02'
        NameCol = 'Cluster'

    df = df.query('Type == "o"') #Only open clusters
    crossmatch = crossmatch.query('SourceCat == @sourcecat') #Crossmatch with the source catalog of the named catalogue
    crossmatch = crossmatch #Drop duplicates

    hunt_xm = pd.merge(crossmatch, df, on='ID', how='inner') #Crossmatched clusters
    hunt_xm.drop_duplicates(subset = ['ID'], keep='first', inplace=True)

    allnames = hunt_xm.assign(synonym = hunt_xm['AllNames'].str.split(',')).explode('synonym') #Create AllNames column with synonyms of the OCs
    allnames = allnames.add_suffix('_hunt') #Add suffix to the columns to avoid confusion with the other table

    df_matched = pd.merge(allnames, df_lit, left_on='synonym_hunt', right_on=NameCol, how='outer', indicator=True) #Merge the crossmatched table with the named catalogue
    
    superclusters = df_matched.groupby(['ID_hunt', 'Name_hunt'])[NameCol].agg(list).reset_index() # multiple lit clusters link to one hunt cluster
    subclusters = df_matched.groupby([NameCol])[['ID_hunt', 'Name_hunt']].agg(list).reset_index() # multiple hunt clusters link to one lit cluster

    matched = df_matched.query('_merge == "both"') #Matched clusters
    not_matched = finmerge.query('_merge == "left_only"') #Not matched clusters

    hunt_matched = matched.filter(regex='_h$').drop(columns=['synonym_h']) # Matched clusters in Hunt (I)
    # remove _h from the column names
    hunt_matched.columns = hunt_matched.columns.str.replace('_h', '')

    lit_matched = matched[df_lit.columns] # Matched clusters in literature (II)
    lit_not_matched = not_matched[df_lit.columns] # Not matched clusters in literature (III)

    print(f'Matched {len(hunt_matched)} + Not matched {len(lit_not_matched)} = Total {len(df_lit)}')
    return hunt_matched, lit_matched, lit_not_matched, superclusters, subclusters

hunt_matched, lit_matched, lit_not_matched, superclusters, subclusters = datahandler(cantat)


NameError: name 'xmatch' is not defined

In [ ]:
hunt_matched[hunt_matched['Cluster'].duplicated(keep=False)].sort_values('Cluster')[['ID_hunt','Name_hunt','Cluster','synonym_hunt']]

NameError: name 'hunt_matched' is not defined

In [10]:
subclusters.query('Cluster == "UBC_6"')

NameError: name 'subclusters' is not defined